<a href="https://colab.research.google.com/github/IdjiotSandwiches/face-emotion-recognition/blob/knn-model/machine-learning/knn-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dagshub --quiet
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/

In [1]:
import cv2 as cv
import numpy as np
import mlflow
import dagshub
import os
import pathlib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.decomposition import PCA

In [2]:
PATH = pathlib.Path('C:\\Users\\vinar\\Downloads\\facial-emotion-recognition-augmented')
LABELS = os.listdir(PATH)

In [8]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='face-emotion-recognition', mlflow=True)

Accessing as IdjiotSandwiches

Initialized MLflow to track repo "IdjiotSandwiches/face-emotion-recognition"

Repository IdjiotSandwiches/face-emotion-recognition initialized!

In [37]:
K_SIZE = (31,31)
SIGMA = 4
THETA_RANGE = np.arange(0, np.pi, np.pi/128)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0
FLOATING_POINT = cv.CV_64F
IMAGE_SIZE = (128,128)
N_COMPONENTS = 20
BLUR = (5,5)

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

KERNELS = [cv.getGaborKernel(**gabor_params, theta=theta) for theta in THETA_RANGE]

In [38]:
def open_dataset(dir):
  images, labels = [], []

  for i, label in enumerate(LABELS):
    path = f'{dir}/{label}'
    for j, img_path in enumerate(os.listdir(path)):
      img = cv.imread(f'{path}/{img_path}', cv.IMREAD_GRAYSCALE)
      img = cv.resize(img, IMAGE_SIZE)
      img = cv.GaussianBlur(img,BLUR,0)
      img = cv.equalizeHist(img)
      img = img / 255.0
      img = gabor_filter(img)

      img = img.reshape(img.shape[0],-1)
      pca = PCA(n_components=N_COMPONENTS)
      img = pca.fit_transform(img)

      images.append(img.reshape(-1))
      labels.append(i)

  return np.array(images), np.array(labels)

In [11]:
def gabor_filter(img):
  return np.array([cv.filter2D(img, FLOATING_POINT, kernel) for kernel in KERNELS])

In [39]:
def save_filtered_img(images, labels):
  folder_path = f'C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs'
  os.makedirs(folder_path, exist_ok=True)

  np.save(f'{folder_path}/images.npy', images)
  np.save(f'{folder_path}/labels.npy', labels)

  print('Ok!')

In [40]:
from joblib import Parallel, delayed

def process_image(img_path, label, dir):
    """
    Process a single image: read, preprocess, and apply PCA.
    """
    path = f'{dir}/{label}'
    img = cv.imread(f'{path}/{img_path}', cv.IMREAD_GRAYSCALE)
    img = cv.resize(img, IMAGE_SIZE)
    img = cv.GaussianBlur(img, (5, 5), 0)
    img = cv.equalizeHist(img)
    img = img / 255.0
    img = gabor_filter(img)

    img = img.reshape(img.shape[0], -1)
    pca = PCA(n_components=N_COMPONENTS)
    img = pca.fit_transform(img)

    return img.reshape(-1), label


def open_dataset(dir):
    """
    Open dataset, load images, preprocess, and return images and labels using parallel processing.
    """
    # Use Parallel from joblib to process images in parallel
    results = Parallel(n_jobs=4)(delayed(process_image)(img_path, label, dir)
                                   for label in LABELS
                                   for img_path in os.listdir(f'{dir}/{label}'))

    # Unzip the results into images and labels
    images, labels = zip(*results)

    return np.array(images), np.array(labels)

In [41]:
images, labels = open_dataset(PATH)
save_filtered_img(images, labels)

Ok!


## **Testing**

In [66]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\labels.npy')

N_NEIGHBORS = 1
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'ball_tree'
TEST_SIZE = 0.5
RANDOM_STATE = 42
N_SPLITS = 10
SHUFFLE = True

model_params = {
  'n_neighbors': N_NEIGHBORS,
  'weights': WEIGHTS,
  'metric': METRIC,
  'algorithm': ALGORITHM
}

calib_params = {
  'test_size': TEST_SIZE,
  'random_state': RANDOM_STATE
}

kfold_params = {
    'n_splits': N_SPLITS,
    'shuffle': SHUFFLE
}

skf = StratifiedKFold(**kfold_params)
i = 0

for train_idx, test_idx in skf.split(images, labels):
  X_train, X_test = images[train_idx], images[test_idx]
  y_train, y_test = labels[train_idx], labels[test_idx]

  X_calib, X_test, y_calib, y_test = train_test_split(X_test, y_test, **calib_params)

  model = KNeighborsClassifier(**model_params)
  model.fit(X_train, y_train)

  calib_model = CalibratedClassifierCV(model, cv="prefit")
  calib_model.fit(X_calib, y_calib)

  predict = calib_model.predict(X_test)

  metrics = {
    'accuracy': accuracy_score(y_test, predict),
    'precision': precision_score(y_test, predict, average='macro'),
    'recall': recall_score(y_test, predict, average='macro'),
    'f1': f1_score(y_test, predict, average='macro'),
  }

  print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")
  print(classification_report(y_test, predict))

  mlflow.set_experiment("KNN_Model k=1 n_component=20")
  mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow")

  with mlflow.start_run(run_name=f'KNN StratifiedKFold={i}'):
    mlflow.log_params(gabor_params)
    mlflow.log_params(calib_params)
    mlflow.log_params(model_params)
    mlflow.log_params(kfold_params)
    mlflow.log_param('floating_point', FLOATING_POINT)
    mlflow.log_param('image_size', IMAGE_SIZE)
    mlflow.log_param('PCA_n_components', N_COMPONENTS)
    mlflow.log_param('gaussian_blur', BLUR)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
        sk_model=calib_model,
        artifact_path='KNN Model',
        input_example=X_train[:1]
    )
  i = i + 1

Accuracy: 0.5495798319327732
Precision: 0.5523258848837649
Recall: 0.553773966973019
F1 Score: 0.5503296214681923
              precision    recall  f1-score   support

       angry       0.53      0.55      0.54        84
     disgust       0.60      0.57      0.58        92
        fear       0.63      0.51      0.56        89
       happy       0.47      0.47      0.47        92
     neutral       0.52      0.55      0.54        87
         sad       0.55      0.51      0.53        78
    surprise       0.56      0.73      0.63        73

    accuracy                           0.55       595
   macro avg       0.55      0.55      0.55       595
weighted avg       0.55      0.55      0.55       595



2024/12/13 16:59:46 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model k=1 n_component=20' does not exist. Creating a new experiment.


🏃 View run KNN StratifiedKFold=0 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6/runs/f549550babdd48178cdd17a79223a20a
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6
Accuracy: 0.5546218487394958
Precision: 0.5471447702307616
Recall: 0.5520005112559041
F1 Score: 0.546965475310057
              precision    recall  f1-score   support

       angry       0.48      0.39      0.43        84
     disgust       0.66      0.75      0.70        92
        fear       0.59      0.61      0.60        89
       happy       0.54      0.41      0.47        92
     neutral       0.61      0.66      0.63        87
         sad       0.47      0.53      0.49        78
    surprise       0.49      0.52      0.50        73

    accuracy                           0.55       595
   macro avg       0.55      0.55      0.55       595
weighted avg       0.55      0.55      0.55       595



🏃 View run KNN StratifiedKFold=1 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6/runs/12fdb46af4dc4bba8472f624c1fb6bce
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6
Accuracy: 0.5630252100840336
Precision: 0.5655523276592712
Recall: 0.5624382008214516
F1 Score: 0.5592134697956516
              precision    recall  f1-score   support

       angry       0.53      0.63      0.58        84
     disgust       0.69      0.64      0.66        92
        fear       0.66      0.64      0.65        89
       happy       0.47      0.38      0.42        92
     neutral       0.55      0.55      0.55        87
         sad       0.48      0.65      0.55        78
    surprise       0.58      0.44      0.50        73

    accuracy                           0.56       595
   macro avg       0.57      0.56      0.56       595
weighted avg       0.57      0.56      0.56       595



🏃 View run KNN StratifiedKFold=2 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6/runs/36fd883c00cb4df98497ef82f20ba8e0
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6
Accuracy: 0.5579831932773109
Precision: 0.5571879817753216
Recall: 0.5573738214683294
F1 Score: 0.5558436824722663
              precision    recall  f1-score   support

       angry       0.45      0.48      0.47        84
     disgust       0.66      0.66      0.66        92
        fear       0.60      0.60      0.60        89
       happy       0.55      0.45      0.49        92
     neutral       0.54      0.63      0.58        87
         sad       0.52      0.50      0.51        78
    surprise       0.58      0.59      0.59        73

    accuracy                           0.56       595
   macro avg       0.56      0.56      0.56       595
weighted avg       0.56      0.56      0.56       595



...

🏃 View run KNN StratifiedKFold=5 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6/runs/a4671333da4946baa8566fadb0e4b2b8
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6
Accuracy: 0.5512605042016807
Precision: 0.5472649380720581
Recall: 0.5466349740224863
F1 Score: 0.544120272636018
              precision    recall  f1-score   support

       angry       0.53      0.60      0.56        84
     disgust       0.65      0.67      0.66        92
        fear       0.54      0.55      0.55        89
       happy       0.52      0.46      0.49        92
     neutral       0.55      0.69      0.61        87
         sad       0.51      0.44      0.47        78
    surprise       0.53      0.42      0.47        73

    accuracy                           0.55       595
   macro avg       0.55      0.55      0.54       595
weighted avg       0.55      0.55      0.55       595



🏃 View run KNN StratifiedKFold=6 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6/runs/cdc1eb1135bf4e6b8b5ab2d3dc76163b
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6
Accuracy: 0.5563025210084034
Precision: 0.5572933747682663
Recall: 0.5562971790764851
F1 Score: 0.5561464128759568
              precision    recall  f1-score   support

       angry       0.48      0.50      0.49        84
     disgust       0.69      0.62      0.65        92
        fear       0.62      0.61      0.61        89
       happy       0.48      0.45      0.46        92
     neutral       0.55      0.61      0.58        87
         sad       0.52      0.55      0.53        78
    surprise       0.57      0.56      0.57        73

    accuracy                           0.56       595
   macro avg       0.56      0.56      0.56       595
weighted avg       0.56      0.56      0.56       595



🏃 View run KNN StratifiedKFold=7 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6/runs/922718e2ed954112b353c3fb14494236
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6
Accuracy: 0.5495798319327732
Precision: 0.5497367820767797
Recall: 0.5469688477151878
F1 Score: 0.5452142763205569
              precision    recall  f1-score   support

       angry       0.45      0.44      0.45        84
     disgust       0.67      0.73      0.70        92
        fear       0.63      0.61      0.62        89
       happy       0.61      0.45      0.52        92
     neutral       0.54      0.56      0.55        87
         sad       0.43      0.55      0.49        78
    surprise       0.51      0.49      0.50        73

    accuracy                           0.55       595
   macro avg       0.55      0.55      0.55       595
weighted avg       0.55      0.55      0.55       595



🏃 View run KNN StratifiedKFold=8 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6/runs/e270216edddb411a8a3c35f011bb40cb
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6
Accuracy: 0.5563025210084034
Precision: 0.5561656099276815
Recall: 0.5565936355304336
F1 Score: 0.553987620354758
              precision    recall  f1-score   support

       angry       0.56      0.48      0.52        84
     disgust       0.58      0.73      0.64        92
        fear       0.66      0.63      0.64        89
       happy       0.42      0.37      0.39        92
     neutral       0.51      0.55      0.53        87
         sad       0.59      0.54      0.56        78
    surprise       0.57      0.60      0.59        73

    accuracy                           0.56       595
   macro avg       0.56      0.56      0.55       595
weighted avg       0.55      0.56      0.55       595



🏃 View run KNN StratifiedKFold=9 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6/runs/3a3464686f044017be6c93f46d0bb163
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/6


## **Testing images shape**

In [63]:
images_ = []
# img = cv.imread(f'facial-emotion-recognition-augmented/disgust/disgust_1022.png', cv.IMREAD_GRAYSCALE)
img = cv.imread(f'../fear_1.png', cv.IMREAD_GRAYSCALE)
img = cv.resize(img, (128,128))
img = cv.GaussianBlur(img,(5,5),0)
img = cv.equalizeHist(img)
img = img / 255.0
img = gabor_filter(img)
img = np.array(img)
img = img.reshape(img.shape[0],-1)
pca = PCA(n_components=20)
img = pca.fit_transform(img)
img = img.reshape(-1)
images_.append(img)
images_ = np.array(images_)

print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Cumulative explained variance:", np.cumsum(pca.explained_variance_ratio_))

print(images_.shape)

Explained variance ratio: [4.89590654e-01 2.49479773e-01 1.08638827e-01 8.26421203e-02
 2.72293607e-02 2.42706556e-02 7.61720069e-03 6.15644662e-03
 1.69685333e-03 1.32139716e-03 9.79791402e-04 2.32195043e-04
 5.68627687e-05 4.71817939e-05 3.37111663e-05 4.91209970e-06
 1.06612222e-06 6.98185910e-07 2.24070131e-07 5.38947473e-08]
Cumulative explained variance: [0.48959065 0.73907043 0.84770925 0.93035137 0.95758074 0.98185139
 0.98946859 0.99562504 0.99732189 0.99864329 0.99962308 0.99985528
 0.99991214 0.99995932 0.99999303 0.99999794 0.99999901 0.99999971
 0.99999993 0.99999999]
(1, 2560)


In [45]:
logged_model = 'runs:/7bff0cfb37d54022af5667731d530b73/KNN Model'
model = mlflow.pyfunc.load_model(logged_model)

In [64]:
prediction = model.predict(images_)
for p in prediction:
  print(p)

angry


## **Tuning**

### **Without PCA**

In [ ]:
images = np.load('/content/drive/MyDrive/gabor-filtered-imgs/images.npy')
labels = np.load('/content/drive/MyDrive/gabor-filtered-imgs/labels.npy')

N_NEIGHBORS = [3,5,7,9,11]
WEIGHTS = ['uniform', 'distance']
METRIC = ['minkowski', 'euclidean', 'manhattan']
ALGORITHM = ['auto', 'ball_tree', 'kd_tree', 'brute']
TEST_SIZE = 0.6
RANDOM_STATE = 42
N_SPLITS = 15
SHUFFLE = True

calib_params = {
  'test_size': TEST_SIZE,
  'random_state': RANDOM_STATE
}

for n in N_NEIGHBORS:
  for w in WEIGHTS:
    for m in METRIC:
      for a in ALGORITHM:
        model_params = {
          'n_neighbors': n,
          'weights': w,
          'metric': m,
          'algorithm': a
        }

        X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
        X_calib, X_test, y_calib, y_test = train_test_split(X_test, y_test, **calib_params)

        model = KNeighborsClassifier(**model_params)
        model.fit(X_train, y_train)

        calib_model = CalibratedClassifierCV(model, cv="prefit")
        calib_model.fit(X_calib, y_calib)

        predict = calib_model.predict(X_test)

        metrics = {
          'accuracy': accuracy_score(y_test, predict),
          'precision': precision_score(y_test, predict, average='macro'),
          'recall': recall_score(y_test, predict, average='macro'),
          'f1': f1_score(y_test, predict, average='macro'),
        }

        print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")
        print(classification_report(y_test, predict))

        mlflow.set_experiment("KNN_Model Tuning Model without using PCA")
        mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

        with mlflow.start_run():
          mlflow.log_params(gabor_params)
          mlflow.log_params(calib_params)
          mlflow.log_params(model_params)
          # mlflow.log_params(kfold_params)
          mlflow.log_param('floating_point', FLOATING_POINT)
          mlflow.log_param('image_size', IMAGE_SIZE)
          mlflow.log_param('PCA_n_components', N_COMPONENTS)
          mlflow.log_metrics(metrics)
          mlflow.sklearn.log_model(
              sk_model=calib_model,
              artifact_path='KNN Model',
                input_example=X_train[:1]
          )

Accuracy: 0.2673434856175973
Precision: 0.2725667470879056
Recall: 0.274931070605638
F1 Score: 0.2566980461274672
              precision    recall  f1-score   support

           0       0.33      0.19      0.24       177
           1       0.28      0.44      0.34       153
           2       0.25      0.48      0.33       153
           3       0.25      0.25      0.25       189
           4       0.28      0.13      0.18       182
           5       0.26      0.29      0.27       161
           6       0.26      0.14      0.19       167

    accuracy                           0.27      1182
   macro avg       0.27      0.27      0.26      1182
weighted avg       0.27      0.27      0.25      1182

🏃 View run placid-bat-562 at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/14/runs/6d5a1239cd3d4c0da9fa00e94241104f
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/14
Accuracy: 0.2673434856175973
Precision: 0.2725667470879056
Re

### **Using calibration**

In [ ]:
N_NEIGHBORS = 9
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'ball_tree'

TEST_SIZE = 0.3
CALIB_SIZE = 0.5
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

calib_params = {
    'test_size': CALIB_SIZE,
    'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params)
X_test, X_calib, y_test, y_calib = train_test_split(X_test, y_test, **calib_params)

model_params = {
  'n_neighbors': N_NEIGHBORS,
  'weights': WEIGHTS,
  'metric': METRIC,
  'algorithm': ALGORITHM
}

model = KNeighborsClassifier(**model_params)
model.fit(X_train, y_train)

calib_model = CalibratedClassifierCV(model, cv="prefit")
calib_model.fit(X_calib, y_calib)

predict = calib_model.predict(X_test)

metrics = {
  'accuracy': accuracy_score(y_test, predict),
  'precision': precision_score(y_test, predict, average='macro'),
  'recall': recall_score(y_test, predict, average='macro'),
  'f1': f1_score(y_test, predict, average='macro'),
}

print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")
print(classification_report(y_test, predict))

mlflow.set_experiment("KNN_Model using calibration")
mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

with mlflow.start_run(run_name=f'KNN sigma=5'):
  mlflow.log_params(gabor_params)
  mlflow.log_params(split_params)
  mlflow.log_param('calib_test_size', calib_params['test_size'])
  mlflow.log_param('calib_random_state', calib_params['random_state'])
  mlflow.log_params(model_params)
  mlflow.log_metrics(metrics)
  mlflow.sklearn.log_model(
      sk_model=calib_model,
      artifact_path='KNN Model',
      input_example=X_train[:1]
  )

Accuracy: 0.41271820448877805
Precision: 0.35505141285443115
Recall: 0.33373441164794554
F1 Score: 0.3324115318106068


2024/12/04 03:16:05 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model using calibration' does not exist. Creating a new experiment.


🏃 View run KNN sigma=5 at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/8/runs/c1b2dcc5917e455fa84e0b57000e5062
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/8


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

### **Tuning k Value**

In [65]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\labels.npy')

N_NEIGHBORS = range(1,21)
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'ball_tree'
TEST_SIZE = 0.5
RANDOM_STATE = 42
N_SPLITS = 10
SHUFFLE = True

X_train, X_test, y_train, y_test = train_test_split(images, labels, random_state=RANDOM_STATE, test_size=0.2)

def evaluate_knn(k):
    model_params = {
        'n_neighbors': k,
        'weights': WEIGHTS,
        'metric': METRIC,
        'algorithm': ALGORITHM
    }
    model = KNeighborsClassifier(**model_params)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return cv_scores.mean()

# Parallel execution
scores = Parallel(n_jobs=4)(delayed(evaluate_knn)(k) for k in N_NEIGHBORS)

# Find optimal k
optimal_k = N_NEIGHBORS[scores.index(max(scores))]
print(f"Optimal k: {optimal_k}")

Optimal k: 1


## **Prediction result**

In [52]:
not_same = []

for p, y in zip(predict, y_test):
  if(p != y):
    not_same.append(p)

print(len(not_same))

1120
